In [57]:
import os

#from IPython.display import set_matplotlib_formats
import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import scipy

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
from sklearn.model_selection import train_test_split
numpyro.set_platform("cpu")

In [2]:
# Try to implement categories as one hot encoding
# start, end, color, form, size
start = jnp.array([1, 0, 0, 0, 0])
end = jnp.array([0, 1, 0, 0, 0])
color = jnp.array([0, 0, 1, 0, 0])
form = jnp.array([0, 0, 0, 1, 0])
size = jnp.array([0, 0, 0, 0, 1])

vocab = ["start", "end", "color", "form", "size"]

# Define the vocabulary
vocab = ['start', 'end', 'color', 'form', 'size']

# Create one-hot vectors for each vocabulary item
vocab_size = len(vocab)
one_hot_vectors = jnp.eye(vocab_size)

# Map each word to its one-hot vector
vocab_to_one_hot = {word: one_hot_vectors[i] for i, word in enumerate(vocab)}

# Display the one-hot vectors
for word, one_hot in vocab_to_one_hot.items():
    print(f"{word}: {one_hot}")

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
2024-07-15 15:03:20.254601: W pjrt_plugin/src/mps_client.cc:563] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

start: [1. 0. 0. 0. 0.]
end: [0. 1. 0. 0. 0.]
color: [0. 0. 1. 0. 0.]
form: [0. 0. 0. 1. 0.]
size: [0. 0. 0. 0. 1.]


In [4]:
# Define a simple language model using one hot vectors
# Start-firstnode-secondnode-end
# At start, equal probability of color, form, size followed by start symbol
# At first node, equal probability of color, form, size and end symbol
# At second node, equal probability of color, form, size and end symbol
# At end, only end symbol

vocab = ['color', 'form', 'size', 'SOS', 'EOS']

def vocab_to_one_hot(vocab):
    """
    Input: vocab - list of words in the vocabulary
    Output: dictionary mapping each word to its one-hot vector
    """
    # Create one-hot vectors for each vocabulary item
    vocab_size = len(vocab)
    one_hot_vectors = jnp.eye(vocab_size)
    
    # Map each word to its one-hot vector
    vocab_to_one_hot = {word: one_hot_vectors[i] for i, word in enumerate(vocab)}
    
    return vocab_to_one_hot

partial_input = ""
def PCFG(partial_input, node):
    # Define the recursive end state
    if node == 'end':
        return vocab_to_one_hot['EOS']
    else:
        PCFG(partial_input, node)
    
    pass
    # Define the transition probabilities



def language_model(vocab):
    """
    Input: vocab - dictionary of one hot vectors for each word in the vocabulary
    Output: prior probabilities of each full utterances
    Function: Implement a simple PCFG language model
    Description for the PCFG:
    1. Starting with the SOS symbol
    2. Equal probability of color, form, size on first node
    3. If color is given from the first node, equal probability of form, size and EOS on second node. Same for if form or size is given
    4. If both color and form are given from the first node, equal probability of size and EOS on third node. Same for if color and size or form and size are given
    5. Whatever EOS is given, the sentence ends
    """
    # Define vocabulary
    
    
    # Create one-hot vectors for each vocabulary item
    vocab_vectors = jnp.eye(len(vocab))
    
    # Map each word to its one-hot vector
    word_to_vector = {word: vocab_vectors[i] for i, word in enumerate(vocab)}
    
    # Transition probabilities
    # At start: equal probability for color, form, size
    start_probs = jnp.array([0, 0, 0, 1, 0])

    # At first node: equal probability for color, form, size, and end
    first_node_probs = jnp.array([1/3, 1/3, 1/3, 0, 0])
    
    # At second node: equal probability for color, form, size, and end
    second_node_probs = jnp.array([1/4, 1/4, 1/4, 0, 1/4])
    

    # At end: only end symbol
    end_probs = jnp.array([0, 0, 0, 0, 1])
    
    # Define the transitions
    transitions = {
        'start': start_probs,
        'firstnode': node_probs,
        'secondnode': node_probs,
        'end': end_probs
    }
    
    # Return the model components
    return word_to_vector, transitions

word_to_vector, transitions = language_model()

print(word_to_vector)
print(transitions)

{'color': Array([1., 0., 0., 0., 0.], dtype=float32), 'form': Array([0., 1., 0., 0., 0.], dtype=float32), 'size': Array([0., 0., 1., 0., 0.], dtype=float32), 'start': Array([0., 0., 0., 1., 0.], dtype=float32), 'end': Array([0., 0., 0., 0., 1.], dtype=float32)}
{'start': Array([0.33333334, 0.33333334, 0.33333334, 0.        , 0.        ],      dtype=float32), 'firstnode': Array([0.25, 0.25, 0.25, 0.  , 0.25], dtype=float32), 'secondnode': Array([0.25, 0.25, 0.25, 0.  , 0.25], dtype=float32), 'end': Array([0, 0, 0, 0, 1], dtype=int32)}


In [61]:
utterances = [
    "D",
    "C",
    "F",
    "CD", #CD
    "CF",
    "DC",
    "DF",
    "FC",
    "FD", #FD
    "DCF",
    "DFC",
    "CDF",
    "CFD", # CFD
    "FCD", # FCD
    "FDC", # FDC
    ]


def utterance_utils(uttrances, biased=False):
    """
    Input: list of utterances
    Output: list of utils scores for each utterance

    Depends on the length of utterances, assign 3 to the ones with length 1, 2 to the ones with length 2 and 1 to the ones with length 3
    Also, allow to costumize the utils scores given the value of the utterances
    If biased = True, assign 0.5 to "CD, FD, CFD, FCD, FDC", assign +1 to utterances staring with "D" on top of the previous rule
    """
    # Calculate the utils scores based on the length of utterances
    utils = [3 if len(utt) == 1 else 2 if len(utt) == 2 else 1 for utt in utterances]

    # Customize the utils scores if biased is True
    if biased:
        biased_utils = [0.5 if utterances[i] == "CD" or 
                        utterances[i] == "FD" or 
                        utterances[i] == "CFD" 
                        or utterances[i] == "FCD" 
                        or utterances[i] == "FDC" else utils[i] for i in range(len(utterances))]
        utils = [1 + biased_utils[i] if utterances[i].startswith("D") else biased_utils[i] for i in range(len(utterances))]

    utils = jnp.array(utils)
    return utils
    
utils = utterance_utils(utterances, biased = True)
print(utils)

def utterance_prior(utils):
    """
    Input: list of utils scores
    Output: list of prior probabilities for each utterance
    """
    prior = jnp.exp(utils) / jnp.sum(jnp.exp(utils))
    return prior

utt_prior = utterance_prior(utils)

def normalize(arr, axis=1):
    """
    Normalize arr along axis
    """
    return arr / jnp.sum(arr, axis=axis, keepdims=True)

def uniform_state_prior(nobj=6):
    """
    Input: number of objects
    Output: list of prior probabilities for each object
    """
    prior=normalize(jnp.ones((2,nobj)))
    return prior

stt_prior = uniform_state_prior()
print(stt_prior)

def get_threshold_kp(states, k=0.5):
    min_val = jnp.min(states[:,0])
    max_val = jnp.max(states[:,0])
    threshold = max_val - k * (max_val - min_val)
    return threshold

def get_size_semval(size,threshold,wf=0.5):
  return 1 - dist.Normal(size - threshold, wf * jnp.sqrt(size ** 2 + threshold ** 2)).cdf(0.0)

def meaning(word, states, state_prior):
    color_semvalue = 0.9
    form_semvalue = 0.95

    if word == "C":
        probs = jnp.where((1. == states[:,1]), color_semvalue, 1 - color_semvalue)
    
    if word == "F":
        probs = jnp.where((1. == states[:,2]), form_semvalue, 1 - form_semvalue)

    if word == "D":
        threshold = get_threshold_kp(states, state_prior[0,:])
        probs = jax.vmap(get_size_semval, in_axes = (0, None))(states[:,0], threshold) # Apply the meaning function for size adjective
    
    return probs
        
def incremental_literal_listener(states):
    # C
    probs_C = meaning("C", states)
    pass




[4.  3.  3.  0.5 2.  3.  3.  2.  0.5 2.  2.  1.  0.5 0.5 0.5]
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]


In [62]:
states_manuell = jnp.array([[10., 1., 1.],
                            [10., 1., 1.],
                            [3., 1., 1.],
                            [3., 1., 0.],
                            [3., 1., 0.],
                            [1., 0., 1.]], dtype=jnp.float32)
print(meaning("D", states_manuell, stt_prior))


XlaRuntimeError: UNKNOWN: /Users/heningwang/Documents/GitHub/numpyro_adjective_modelling/jax-metal/lib/python3.11/site-packages/numpyro/distributions/continuous.py:2049:15: error: failed to legalize operation 'chlo.erfc'
        return ndtr(scaled)
              ^
/Users/heningwang/Documents/GitHub/numpyro_adjective_modelling/jax-metal/lib/python3.11/site-packages/numpyro/distributions/continuous.py:2049:15: note: see current operation: %0 = "chlo.erfc"(%arg0) : (tensor<6x6xf32>) -> tensor<6x6xf32>


In [1]:
import jax.numpy as jnp

def encode_utterances(utterances_list):
    """
    Input: list of strings of 15 categories
    Output: jnp.array of indices
    Given the ordering of utterances, encode the strings into indices.
    """
    # Define the fixed ordering of utterances
    utterances_order = [
        "D", "C", "F", "CD", "CF", "DC", "DF",
        "FC", "FD", "DCF", "DFC", "CDF", "CFD", "FCD", "FDC"
    ]
    
    # Create a dictionary that maps each utterance to its index
    utterance_to_index = {utterance: idx for idx, utterance in enumerate(utterances_order)}
    
    # Encode the input list of strings into indices
    indices = [utterance_to_index[utterance] for utterance in utterances_list]
    
    # Return the indices as a jnp.array
    return jnp.array(indices)

# Example usage
utterances = ['C', 'C', 'C', 'DCF', 'CF']

encoded_indices = encode_utterances(utterances)
print(encoded_indices)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!


Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

[1 1 1 9 4]


2024-07-18 17:47:42.318667: W pjrt_plugin/src/mps_client.cc:563] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
